In [10]:
# Import Drive API and authenticate.
from google.colab import drive

# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [11]:
! pip uninstall preprocessor

In [33]:
! pip uninstall tweet-preprocessor

Uninstalling tweet-preprocessor-0.6.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/preprocessor/api.py
    /usr/local/lib/python3.6/dist-packages/preprocessor/defines.py
    /usr/local/lib/python3.6/dist-packages/preprocessor/enum.py
    /usr/local/lib/python3.6/dist-packages/preprocessor/parse.py
    /usr/local/lib/python3.6/dist-packages/preprocessor/preprocess.py
    /usr/local/lib/python3.6/dist-packages/preprocessor/utils.py
    /usr/local/lib/python3.6/dist-packages/tests/*
    /usr/local/lib/python3.6/dist-packages/tweet_preprocessor-0.6.0.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/tests/download_media.py
    /usr/local/lib/python3.6/dist-packages/tests/test_AudioClips.py
    /usr/local/lib/python3.6/dist-packages/tests/test_ImageSequenceClip.py
    /usr/local/lib/python3.6/dist-packages/tests/test_PR.py
    /usr/local/lib/python3.6/dist-packages/tests/test_TextClip.py
    /usr/local/lib/python3.6/dist-pack

In [12]:
! pip install tweet-preprocessor

In [1]:
! pip install spacy
! pip install emoji
! pip install spacy_cld

In [0]:
import pandas as pd
import numpy as np
import torch as tc
import preprocessor as p
import os
import emoji
import re
import string
import spacy
from spacy_cld import LanguageDetector
from sklearn.model_selection import train_test_split

In [0]:
def id2text(df, x):
    row = df[df.tweet_id == int(x)]
    return '' if (len(row) == 0) else row.iloc[0]['text']


def clean1(tweet):
    p.set_options(p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER)
    clean = p.clean(tweet)

    p.set_options(p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER, p.OPT.HASHTAG, p.OPT.URL)
    clean = p.tokenize(clean)
    clean = re.sub(r'\$HASHTAG\$', '<hashtag>', clean)
    clean = re.sub(r'\$URL\$', '<url>', clean)

    clean = re.sub(emoji.get_emoji_regexp(), '', clean)

    clean = re.sub(u'\ufe0f', '', clean)
    clean = re.sub(r' +', ' ', clean)

    clean = re.sub(r'&gt;', '>', clean)
    clean = re.sub(r'&lt;', '<', clean)
    clean = clean.strip()

    return clean


def clean2(nlp, x):
    doc = nlp(x)
    return x if doc._.languages and ('en' in doc._.language_scores and doc._.language_scores['en'] >= 0.80) else ''


def question_answer(df, nlp, author_id):

    services = df[df.author_id == author_id]
    services = services[~services.in_response_to_tweet_id.isnull()]

    services = services[['author_id', 'text', 'in_response_to_tweet_id']]

    services.loc[:, 'in_response_to_tweet_id'] = services.in_response_to_tweet_id.apply(lambda x: id2text(df, x))

    services.rename(columns={'author_id': 'author_id', 'text': 'answer', 'in_response_to_tweet_id': 'question'},
                           inplace=True)
    services = services[['author_id', 'question', 'answer']]

    services.loc[:, 'question'] = services.question.apply(clean1)
    services.loc[:, 'answer'] = services.answer.apply(clean1)

    services.loc[:, 'question'] = services.question.apply(lambda x: clean2(nlp, x))
    services.loc[:, 'answer'] = services.answer.apply(lambda x: clean2(nlp, x))
    services = services[~(services.question == '') & ~(services.answer == '')]
    services = services[~(services.question == '.') & ~(services.answer == '.')]

    return services

In [0]:
def main():
  with open('/gdrive/My Drive/Project/Data/twcs.csv', 'r') as f:
    df = pd.read_csv(f)

  nlp = spacy.load('en')
  #nlp.add_pipe(LanguageDetector())

  author_ids = ['AppleSupport', 'SpotifyCares']
  random_state = 287

  for author_id in author_ids:

    print(author_id)
    dataset = question_answer(df, nlp, author_id)

    author_id = author_id.lower()
    with open('/gdrive/My Drive/Project/Data/' + author_id + '.csv', 'w') as f:
      dataset.to_csv(f)

    train, rest = train_test_split(dataset, test_size=0.2, random_state=random_state)
    val, test = train_test_split(rest, test_size=0.5, random_state=random_state)

    with open('/gdrive/My Drive/Project/Data/' + author_id + '_train' + '.csv', 'w') as f:
      train.to_csv(f)

    with open('/gdrive/My Drive/Project/Data/' + author_id + '_val' + '.csv', 'w') as f:
      val.to_csv(f)

    with open('/gdrive/My Drive/Project/Data/' + author_id + '_test' + '.csv', 'w') as f:
      test.to_csv(f)

In [0]:
!cp '/gdrive/My Drive/Project/Data/twcs.csv' .

In [34]:
main()

AppleSupport
SpotifyCares
